In [2]:
import pandas as pd
import numpy as np

import os

#os.listdir(".")
#credit = pd.read_csv("Credit_DataSet.csv")

# TODO: READ THE PROCESSED CSV INSTEAD (FOR EXAMPLE WITH CONVERTED CURRENCY)!

credit = pd.read_excel("./Credit/Credit_DataSet.xlsx", sheetname="DATA", skiprows=0)

# Quickly select numeric columns

to_keep = ['NBR_EMPLOYEES', 'COUNTRY_RISK_GROWTH_RATE',
       'COUNTRY_RISK_DIVIDEND_YIELD', 'COUNTRY_RISK_PAYOUT_RATIO',
       'VOLATILITY_30D', 'VOLATILITY_180D', 'PCT_CHG_1_YEAR', 'PCT_CHG_6_M',
       'DIVIDEND_YIELD', 'MARKETCAP', 'TOTAL_ASSETS', 'TOTAL_LIABILITIES',
       'CURRENT_ASSETS', 'EBIT', 'RETAINED_EARNINGS', 'SALES', 'SALES_GROWTH',
       'INTEREST_EXPENSES']

X = credit[ to_keep + ["DEFAULT_PROB"] ].dropna()

Y = X["DEFAULT_PROB"].values
X = X[to_keep].values

In [2]:
import pandas as pd
import numpy as np
# READ PREPROCESSED DATA FROM R FOR NN
X_df = pd.read_csv("some_X.csv", usecols=['NBR_EMPLOYEES', 'EMPL_GROWTH',
       'COUNTRY_RISK_GROWTH_RATE', 'COUNTRY_RISK_DIVIDEND_YIELD',
       'COUNTRY_RISK_PAYOUT_RATIO', 'VOLATILITY_30D', 'VOLATILITY_180D',
       'PCT_CHG_1_YEAR', 'PCT_CHG_6_M', 'DIVIDEND_YIELD', 'MARKETCAP',
       'TOTAL_ASSETS', 'TOTAL_LIABILITIES', 'CURRENT_ASSETS', 'EBIT',
       'RETAINED_EARNINGS', 'SALES', 'SALES_GROWTH', 'INTEREST_EXPENSES',
       'WHISTLE_BLOWER_POLICY', 'ETHICS_POLICY', 'BRIBERY_POLICY',
       'PCT_WOMEN_EMPLOYEES, PCT_WOMEN_MGT'])
Y_df = pd.read_csv("some_Y.csv", usecols=[1])

import torch
from torch import nn
from torch.autograd import Variable

In [52]:
def select_most_likely(mlp, loss_fn, x_row, y_row):
    candidates = [] # TODO: For every -1 in row in the specific columns, try 0 and 1
    
    bad_values = {'WHISTLE_BLOWER_POLICY': 0,
                            'ETHICS_POLICY': 0,
                            'BRIBERY_POLICY': 0,
                 'PCT_WOMEN_EMPLOYEES':-1,
                  'PCT_WOMEN_MGT':-1}

    possible_values_dict = {'WHISTLE_BLOWER_POLICY':[[1,0], [0,1]],
                            'ETHICS_POLICY':[[1,0], [0,1]],
                            'BRIBERY_POLICY':[[1,0], [0,1]],
                           'PCT_WOMEN_EMPLOYEES':[0, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90]}
    
    # Make a dictionary for the candidate values that we will expand later
    candidate_dict = {}
    for k, v in x_row.iteritems():
        candidate_dict[k] = []
        
        if (k in bad_values): # We have to encode for the neural network
            if (v == bad_values[k]):
                for i in possible_values_dict[k]:
                    # Directly encode the bad_value
                    candidate_dict[k].append(i)
            else:
                candidate_dict[k] = [possible_values_dict[k][int(v) - 1]] # HACK: IT'S BECAUSE THEY HAVE VALUES 1 AND 2
        else:
            # Just add the value
            candidate_dict[k].append([v])

    best_candidate = None
    lowest_loss = 100000000000

    # In the dictionary we have the different candidates
    for candidate in product(*candidate_dict.values()):
        input = Variable(torch.FloatTensor(np.array(list(chain(*list(candidate)))))).cuda()
        # Forward the candidate through the network        
        y_pred = mlp(input)
        loss = loss_fn(y_pred, y_row)
        if loss < lowest_loss:
            best_candidate = input
    return best_candidate
    
def get_E_step(mlp, loss_fn, X_df, Y_df):
    """ Return a torch CUDA variable of X"""
    df_list = []        
    for (k, j) in zip(X_df.iterrows(), Y_df.iterrows()):
        (_, x) = k
        (_, y) = j
        df_list.append(select_most_likely(model, loss_fn, x, Variable(torch.FloatTensor(y.values)).cuda()))
    return torch.stack(df_list)


In [105]:
def get_M_step(mlp, loss_fn, X, Y, N_iter):
    learning_rate = 1e-5
    for t in range(N_iter):
        y_pred = model(X)
        loss = loss_fn(y_pred, Y)
        if (t % (N_iter / 2) == 0): print(loss)
        model.zero_grad()
        loss.backward()
        for param in model.parameters():
            param.data -= learning_rate * param.grad.data
    return mlp, loss

def EM_algorithm(X_df, Y_df, N_hidden=10, N_iter=10000):
    
    X = Variable(torch.FloatTensor(X_df.values)).cuda()
    Y = Variable(torch.FloatTensor(Y_df.values)).cuda() # Always stays the same

    # Make an initial model
    print("Making model")
    model = torch.nn.Sequential(
        torch.nn.Linear(X.shape[1] + 3, N_hidden),
        torch.nn.ReLU(),
        #torch.nn.Sigmoid(),
        torch.nn.Linear(N_hidden, Y.shape[1]),
        torch.nn.Sigmoid() # If it doesn't work, look here
    )

    model = model.cuda()
    loss_fn = torch.nn.MSELoss(size_average=False)
    
    previous_loss = 10000000
    # Init
    for i in range(100):
        # Get the expected data
        print("Doing iteration {} E step: finding most likely data".format(i + 1))
        X = get_E_step(model, loss_fn, X_df, Y_df)
        # TODO: Check if the predicted data is the same still?
        # Train the model for some iterations
        print("Doing iteration {} M step: training neural network".format(i + 1))
        learning_rate = 1e-3
        for t in range(N_iter):
            y_pred = model(X)
            loss = loss_fn(y_pred, Y)
            if (t % (N_iter / 2) == 0): print(loss)
            model.zero_grad()
            loss.backward()
            for param in model.parameters():
                param.data -= learning_rate * param.grad.data

        if (-loss + previous_loss)<0.0001: 
            print("Loss not decreasing: exit")
            break
        previous_loss = loss
    return X, model
    

In [182]:
X_final, mlp = EM_algorithm(X_df, Y_df, N_hidden=80, N_iter=10000)

Making model
Doing iteration 1 E step: finding most likely data
Doing iteration 1 M step: training neural network
Variable containing:
 815.6836
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 0.5089
[torch.cuda.FloatTensor of size () (GPU 0)]

Doing iteration 2 E step: finding most likely data
Doing iteration 2 M step: training neural network
Variable containing:
 0.4640
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 0.4480
[torch.cuda.FloatTensor of size () (GPU 0)]

Doing iteration 3 E step: finding most likely data
Doing iteration 3 M step: training neural network
Variable containing:
 0.4295
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 0.4196
[torch.cuda.FloatTensor of size () (GPU 0)]

Doing iteration 4 E step: finding most likely data
Doing iteration 4 M step: training neural network
Variable containing:
 0.4106
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 0.4026
[torch.cuda.FloatTensor of size ()

Variable containing:
 0.1970
[torch.cuda.FloatTensor of size () (GPU 0)]

Doing iteration 34 E step: finding most likely data
Doing iteration 34 M step: training neural network
Variable containing:
 0.1960
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 0.1949
[torch.cuda.FloatTensor of size () (GPU 0)]

Doing iteration 35 E step: finding most likely data
Doing iteration 35 M step: training neural network
Variable containing:
 0.1938
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 0.1928
[torch.cuda.FloatTensor of size () (GPU 0)]

Doing iteration 36 E step: finding most likely data
Doing iteration 36 M step: training neural network
Variable containing:
 0.1919
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 0.1910
[torch.cuda.FloatTensor of size () (GPU 0)]

Doing iteration 37 E step: finding most likely data
Doing iteration 37 M step: training neural network
Variable containing:
 0.1900
[torch.cuda.FloatTensor of size () (GPU 0

Variable containing:
 0.1530
[torch.cuda.FloatTensor of size () (GPU 0)]

Doing iteration 67 E step: finding most likely data
Doing iteration 67 M step: training neural network
Variable containing:
 0.1526
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 0.1522
[torch.cuda.FloatTensor of size () (GPU 0)]

Doing iteration 68 E step: finding most likely data
Doing iteration 68 M step: training neural network
Variable containing:
 0.1518
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 0.1514
[torch.cuda.FloatTensor of size () (GPU 0)]

Doing iteration 69 E step: finding most likely data
Doing iteration 69 M step: training neural network
Variable containing:
 0.1511
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 0.1507
[torch.cuda.FloatTensor of size () (GPU 0)]

Doing iteration 70 E step: finding most likely data
Doing iteration 70 M step: training neural network
Variable containing:
 0.1504
[torch.cuda.FloatTensor of size () (GPU 0

Variable containing:
 0.1344
[torch.cuda.FloatTensor of size () (GPU 0)]

Doing iteration 100 E step: finding most likely data
Doing iteration 100 M step: training neural network
Variable containing:
 0.1340
[torch.cuda.FloatTensor of size () (GPU 0)]

Variable containing:
 0.1338
[torch.cuda.FloatTensor of size () (GPU 0)]



In [140]:
new_df = X_df.copy()

def two_columns_to_one(bla):
    if bla.equal(torch.FloatTensor([1,0]).cuda()):
        return 1
    elif bla.equal(torch.FloatTensor([0,1]).cuda()):
        return 2
    else:
        print("ERROR")
        return 0


def pytorch_vars_to_df(X_df, X):
    new_df = X_df.copy()
    # Map categorical variables back
    cat = {'WHISTLE_BLOWER_POLICY','ETHICS_POLICY','BRIBERY_POLICY'}
    new_df['WHISTLE_BLOWER_POLICY'] = pd.Series([two_columns_to_one(x) for x in X_final[:, 19:21].data])
    new_df['ETHICS_POLICY'] = pd.Series([two_columns_to_one(x) for x in X_final[:, 21:23].data])
    new_df['BRIBERY_POLICY'] = pd.Series([two_columns_to_one(x) for x in X_final[:, 23:25].data])

    return X_df

last_df = pytorch_vars_to_df(X_df, X_final)

last_df.write_csv("EM_data.csv")

0       1
1       1
2       1
3       1
4       1
5       1
6       1
7       1
8       1
9       1
10      1
11      1
12      1
13      1
14      1
15      1
16      2
17      1
18      1
19      1
20      1
21      1
22      1
23      1
24      1
25      1
26      1
27      1
28      1
29      1
       ..
3903    1
3904    1
3905    1
3906    1
3907    1
3908    1
3909    1
3910    1
3911    1
3912    1
3913    1
3914    2
3915    1
3916    1
3917    1
3918    1
3919    1
3920    1
3921    1
3922    2
3923    1
3924    1
3925    1
3926    1
3927    1
3928    1
3929    1
3930    1
3931    1
3932    1
Length: 3933, dtype: int64

Variable containing:
 1
 0
[torch.cuda.FloatTensor of size (2,) (GPU 0)]